In [28]:
# Import packages
import pandas as pd
import numpy as np
import pycountry

In [23]:
# Import data
sectors = pd.read_csv('/Users/sambickel-barlow/Desktop/PP434/Project/AQUASTAT_sector_update.csv')
sdg_crosswalk = pd.read_csv('/Users/sambickel-barlow/Desktop/PP434/Project/world-regions-sdg-united-nations.csv')

In [29]:
# Function to get the ISO3 code for a specific country
def get_iso3(area):
    if area == 'Bolivia (Plurinational State of)':
        return 'BOL'
    elif area == 'Democratic Republic of the Congo':
        return 'COD'
    elif area == 'Iran (Islamic Republic of)':
        return 'IRN'
    elif area == 'Netherlands (Kingdom of the)':
        return 'NLD'
    elif area == 'Venezuela (Bolivarian Republic of)':
        return 'VEN'
    elif area == 'Niger':
        return 'NER'
    elif area == 'Republic of Korea':
        return 'KOR'
    elif area == 'Republic of Korea':
        return 'KOR'
    else:
        # Attempt to use pycountry to get the ISO3 code
        try:
            return pycountry.countries.search_fuzzy(area)[0].alpha_3
        except:
            return 'Not Found'  # Return 'Not Found' if the country is not found

# Applying the function to the 'Area' column and creating the 'ISO3' column
sectors['ISO3'] = sectors['Area'].apply(get_iso3)

In [34]:
# Merge on ISO3 code
sectors_region = sectors.merge(sdg_crosswalk, how='left', left_on='ISO3', right_on='Code')

In [43]:
# Take only a few columns
sectors_region = sectors_region[['Variable','Value', 'Sustainable Development Goals (SDG) Regions']]

In [44]:
# Get average value for each stat and SDG region
sectors_region_mean = sectors_region.groupby(['Variable', 'Sustainable Development Goals (SDG) Regions'])['Value'].mean().reset_index()

In [58]:
# See unique variables
sectors_region_mean['Variable'].unique()

array(['Agricultural water withdrawal as % of total renewable water resources',
       'SDG 6.4.1. Industrial Water Use Efficiency',
       'SDG 6.4.1. Irrigated Agriculture Water Use Efficiency',
       'SDG 6.4.1. Services Water Use Efficiency',
       'SDG 6.4.1. Water Use Efficiency',
       'SDG 6.4.2. Agricultural Sector Contribution to Water Stress',
       'SDG 6.4.2. Industrial Sector Contribution to Water Stress',
       'SDG 6.4.2. Municipal Sector Contribution to Water Stress',
       'SDG 6.4.2. Water Stress'], dtype=object)

In [47]:
# Subset sector contribution variables
sectors_sub = sectors_region_mean[sectors_region_mean['Variable'].isin(['SDG 6.4.2. Agricultural Sector Contribution to Water Stress' , 'SDG 6.4.2. Industrial Sector Contribution to Water Stress' , 'SDG 6.4.2. Municipal Sector Contribution to Water Stress'])].reset_index()

In [61]:
# Subset water stress total variable
ws_sub = sectors_region_mean[sectors_region_mean[['Sustainable Development Goals (SDG) Regions','Variable']]['Variable'].isin(['SDG 6.4.2. Water Stress'])]

In [76]:
# Remerge to calculate share
sectors_sub_share = sectors_sub.merge(ws_sub, how='left', on='Sustainable Development Goals (SDG) Regions')

In [77]:
# Calculate share
sectors_sub_share['share_of_Water_Stress'] = sectors_sub_share['Value_x'] / sectors_sub_share['Value_y']

In [78]:
# Rename columns
sectors_sub_share = sectors_sub_share.rename(columns={'Value_x' : 'Water_Stress_contrib' , 'Value_y' : 'Water_Stress'})

In [80]:
# Function rename sector contributions
def sectors(sector):
    if sector == 'SDG 6.4.2. Agricultural Sector Contribution to Water Stress':
        return 'Agricultural Sector'
    elif sector == 'SDG 6.4.2. Industrial Sector Contribution to Water Stress':
        return 'Industrial Sector'
    elif sector == 'SDG 6.4.2. Municipal Sector Contribution to Water Stress':
        return 'Municipal Sector'
    else:
        return 'Error'
    

sectors_sub_share['Contribution to Water Stress'] = sectors_sub_share['Variable_x'].apply(sectors)

In [81]:
# Drop extra columns
sectors_sub_share = sectors_sub_share.drop(columns=['Variable_x','Variable_y','index'] , axis=1)

In [82]:
sectors_sub_share

,Sustainable Development Goals (SDG) Regions,Water_Stress_contrib,Water_Stress,share_of_Water_Stress,Contribution to Water Stress
0,Australia and New Zealand,4.175979,6.360190,0.656581,Agricultural Sector
1,Central and Southern Asia,53.115731,60.520274,0.877652,Agricultural Sector
2,Eastern and South-Eastern Asia,15.365034,23.328166,0.658647,Agricultural Sector
3,Europe and Northern America,3.972169,16.132114,0.246227,Agricultural Sector
4,Latin America and the Caribbean,7.399709,13.875443,0.533295,Agricultural Sector
5,Northern Africa and Western Asia,249.715254,386.844233,0.645519,Agricultural Sector
6,Oceania,0.087734,0.214808,0.408431,Agricultural Sector
7,Sub-Saharan Africa,8.873705,12.184286,0.728291,Agricultural Sector
8,Australia and New Zealand,1.399132,6.360190,0.219983,Industrial Sector
9,Central and Southern Asia,2.996171,60.520274,0.049507,Industrial Sector


In [83]:
# Output
sectors_sub_share.to_csv('/Users/sambickel-barlow/Desktop/PP434/Project/AQUASTAT_sector_update_clean.csv')